In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from matplotlib import colors
from random import choice
from experiment_class import Experiment,clean_exp_with_hyphaes
from hyphae_id import clean_and_relabel, get_mother

In [2]:
plate = 13
directory='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[1:24]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]

In [3]:
exp= Experiment(13)
exp.load(dates,local=False,raw=False)

In [7]:
from experiment_class import Node
def clean_and_relabel(exp):
    exp_clean= clean_exp_with_hyphaes(exp)
    equ_class,ambig,connection=resolve_ambiguity(exp_clean.hyphaes)
    new_graph,newposs = relabel_nodes_after_amb(connection,exp_clean.nx_graph,exp_clean.positions)
    exp_clean.nx_graph = new_graph
    exp_clean.positions = newposs
    exp_clean.nodes = []
    labels = {int(node) for g in exp_clean.nx_graph for node in g}
    for label in labels:
        exp_clean.nodes.append(Node(label,exp_clean))
    exp_clean_relabeled= clean_exp_with_hyphaes(exp_clean)
    return(exp_clean_relabeled)

In [ ]:
exp_clean = clean_and_relabel(exp)

before getting hyphaes True
0.0
0.06934812760055478
0.13869625520110956
0.20804438280166435
0.27739251040221913
0.34674063800277394
0.4160887656033287
0.4854368932038835
0.5547850208044383
0.624133148404993
0.6934812760055479
0.7628294036061026
0.8321775312066574
0.9015256588072122
0.970873786407767
after getting hyphaes True
removing  1753  nodes
removing False
before pruning True
after pruning True
before changing nx_graph True
after changing nx_graph True
before getting hyphaes True
0.0
0.07827788649706457
0.15655577299412915
0.23483365949119372
0.3131115459882583
0.3913894324853229
0.46966731898238745
0.547945205479452
0.6262230919765166
0.7045009784735812
0.7827788649706457
0.8610567514677103
0.9393346379647749
after getting hyphaes True
removing  155  nodes
removing False
before pruning True
after pruning True


In [ ]:
get_mother(exp_clean.hyphaes)

In [ ]:
hyphs,gr_inf = save_hyphaes(exp_clean_relabeled)
exp_clean_relabeled.save()